In [1]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

In [2]:
data = pd.read_csv('data/koi_data.csv')
data = data.set_index('kepoi_name')
data['koi_disposition'] = data['koi_disposition'] == 'CONFIRMED'
data.head(5)

,koi_disposition,koi_period,koi_impact,koi_duration,koi_depth,koi_ror,koi_srho,koi_prad,koi_sma,koi_incl,...,koi_fwm_srao,koi_fwm_sdeco,koi_fwm_prao,koi_fwm_pdeco,koi_dicco_mra,koi_dicco_mdec,koi_dicco_msky,koi_dikco_mra,koi_dikco_mdec,koi_dikco_msky
kepoi_name,,,,,,,,,,,,,,,,,,,,,
K00752.01,True,9.48804,0.146,2.95750,615.8,0.02234,3.20796,2.26,0.0853,89.66,...,0.430,0.940,-0.00020,-0.00055,-0.010,0.200,0.200,0.080,0.310,0.320
K00752.02,True,54.41838,0.586,4.50700,874.8,0.02795,3.02368,2.83,0.2734,89.57,...,-0.630,1.230,0.00066,-0.00105,0.390,0.000,0.390,0.490,0.120,0.500
K00754.01,False,1.73695,1.276,2.40641,8079.2,0.38739,0.22080,33.46,0.0267,67.09,...,-0.111,0.002,0.00302,-0.00142,-0.249,0.147,0.289,-0.257,0.099,0.276
K00755.01,True,2.52559,0.701,1.65450,603.3,0.02406,1.98635,2.75,0.0374,85.41,...,-0.010,0.230,0.00008,-0.00007,0.030,-0.090,0.100,0.070,0.020,0.070
K00114.01,False,7.36179,1.169,5.02200,233.7,0.18339,0.00485,39.21,0.0820,60.92,...,-13.450,24.090,0.00303,-0.00555,-4.506,7.710,8.930,-4.537,7.713,8.948


In [3]:
data.describe()

,koi_period,koi_impact,koi_duration,koi_depth,koi_ror,koi_srho,koi_prad,koi_sma,koi_incl,koi_teq,...,koi_fwm_srao,koi_fwm_sdeco,koi_fwm_prao,koi_fwm_pdeco,koi_dicco_mra,koi_dicco_mdec,koi_dicco_msky,koi_dikco_mra,koi_dikco_mdec,koi_dikco_msky
count,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,...,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000
mean,37.032237,0.717106,5.607025,21340.318993,0.235205,3.415370,112.230798,0.158146,81.181413,1143.721069,...,-0.355681,-0.805629,-0.000263,0.000439,-0.049743,-0.087413,1.930251,-0.038402,-0.098738,1.920226
std,88.417985,2.628207,6.962634,66989.808550,2.586213,25.131368,3699.799318,0.241792,16.308839,775.788868,...,10.978677,14.741473,0.065707,0.077519,2.465670,2.746534,3.147553,2.465094,2.734732,3.142764
min,0.306940,0.000000,0.104600,0.800000,0.001290,0.000040,0.080000,0.007200,2.290000,92.000000,...,-275.600000,-397.620000,-4.000000,-0.800000,-21.500000,-75.900000,0.000000,-23.600000,-76.600000,0.000000
25%,2.213962,0.226000,2.500250,176.800000,0.013058,0.176092,1.460000,0.033000,81.930000,615.250000,...,-0.500000,-0.570000,-0.000240,-0.000240,-0.270000,-0.291500,0.128250,-0.265250,-0.320000,0.180000
50%,7.386755,0.610000,3.805500,495.950000,0.024185,0.748045,2.600000,0.073650,87.890000,948.000000,...,0.000000,-0.030000,0.000000,0.000000,0.000000,0.000000,0.460000,-0.007000,-0.018000,0.453000
75%,23.448117,0.923750,6.000750,2120.525000,0.171260,2.267063,21.645000,0.158200,89.520000,1482.000000,...,0.500000,0.450000,0.000260,0.000280,0.230000,0.230000,2.570000,0.226250,0.250000,2.420000
max,1071.232620,100.806000,138.540000,864260.000000,99.870650,918.752390,200346.000000,2.034500,90.000000,9791.000000,...,97.780000,98.780000,1.190000,5.000000,45.680000,27.500000,88.600000,46.570000,31.200000,89.600000


In [4]:
x = data.drop('koi_disposition', axis=1)
y = data['koi_disposition']

In [5]:
def create_pipeline(model):
    pipe = Pipeline([['std', StandardScaler()],
                     ['clf', model]])

    return pipe

In [6]:
def create_model(experiments, model_name, model_obj, model_params={}):
    experiments['model_name'].append(model_name)
    experiments['model_object'].append(model_obj(**model_params))
    experiments['model_params'].append(model_params)
    
    return experiments

In [7]:
experiments = {'model_name': [], 'model_object': [], 'model_params': []}

# Naive Bayes baseline
experiments = create_model(experiments, 'NaiveBayes', GaussianNB)

# Decision Tree experiments
experiments = create_model(experiments, 'DecisionTree', DecisionTreeClassifier, {'max_depth': 5})
experiments = create_model(experiments, 'DecisionTree', DecisionTreeClassifier, {'max_depth': 10})
experiments = create_model(experiments, 'DecisionTree', DecisionTreeClassifier, {'max_depth': 15})
experiments = create_model(experiments, 'DecisionTree', DecisionTreeClassifier, {'max_depth': 20})
experiments = create_model(experiments, 'DecisionTree', DecisionTreeClassifier, {'max_depth': None})

# SVM experiments
experiments = create_model(experiments, 'SVM', SVC, {'kernel': 'linear'})
experiments = create_model(experiments, 'SVM', SVC, {'kernel': 'sigmoid'})
experiments = create_model(experiments, 'SVM', SVC, {'kernel': 'poly', 'degree': 2})
experiments = create_model(experiments, 'SVM', SVC, {'kernel': 'poly', 'degree': 3})
experiments = create_model(experiments, 'SVM', SVC, {'kernel': 'poly', 'degree': 4})
experiments = create_model(experiments, 'SVM', SVC, {'kernel': 'rbf'})

# KNN experiments
experiments = create_model(experiments, 'KNN', KNeighborsClassifier, {'n_neighbors': 1})
experiments = create_model(experiments, 'KNN', KNeighborsClassifier, {'n_neighbors': 2})
experiments = create_model(experiments, 'KNN', KNeighborsClassifier, {'n_neighbors': 5})
experiments = create_model(experiments, 'KNN', KNeighborsClassifier, {'n_neighbors': 10})
experiments = create_model(experiments, 'KNN', KNeighborsClassifier, {'n_neighbors': 50})

# Random Forest experiments
experiments = create_model(experiments, 'RandomForest', RandomForestClassifier, {'n_estimators': 50})
experiments = create_model(experiments, 'RandomForest', RandomForestClassifier, {'n_estimators': 100})
experiments = create_model(experiments, 'RandomForest', RandomForestClassifier, {'n_estimators': 200})
experiments = create_model(experiments, 'RandomForest', RandomForestClassifier, {'n_estimators': 500})

# Gradient Tree Boosting experiments

experiments = create_model(experiments, 'GradientTreeBoosting', GradientBoostingClassifier, {'n_estimators': 50})
experiments = create_model(experiments, 'GradientTreeBoosting', GradientBoostingClassifier, {'n_estimators': 100})
experiments = create_model(experiments, 'GradientTreeBoosting', GradientBoostingClassifier, {'n_estimators': 200})
experiments = create_model(experiments, 'GradientTreeBoosting', GradientBoostingClassifier, {'n_estimators': 500})

In [8]:
experiments['pipeline'] = list(map(lambda model: create_pipeline(model), experiments['model_object']))

In [ ]:
experiments['accuracy_scores'] = list(map(
    lambda pipeline: cross_val_score(pipeline, x, y, cv=5, n_jobs=-1), experiments['pipeline']))

In [ ]:
experiments['accuracy_mean'] = list(map(lambda scores: np.mean(scores), experiments['accuracy_scores']))
experiments['accuracy_std'] = list(map(lambda scores: np.std(scores), experiments['accuracy_scores']))

In [ ]:
results = pd.DataFrame(experiments)
print(results)

In [ ]:
print(results[['model_name', 'model_params', 'accuracy_mean', 'accuracy_std']].sort_values(
    'accuracy_mean', ascending=False))